# Generative AI and Prompt Engineering
## A programme by IISc and TalentSprint
### Assignment 1: RAG - Retrieval Augmented Generation

**(with OpenAI LLMs)**

## Learning Objectives

At the end of the experiment, you will be able to:

1. Load the Documents
2. Splitting the documents into chunks
3. Embedding the chunks and storing them in vector db
4. Retrieving the relevant chunks to the query
 * Addressing Diversity
 * Addressing Specificity
5. Connecting with LLM to get a final grounded answer

## Introduction

> **RAG diagram:**
>
> <img src='https://drive.google.com/uc?id=1sCVvpsmtZEU1WSK1FFGMGHbEjrgtCNLi'>

> **Vector Store and Retrieval:**
>
> <img src='https://drive.google.com/uc?id=1_zX5gtSNrV8Qdx7Nz4_gMR8dCwvxCDS7' width=750px>

> **Embedding Model:**
>
> <img src='https://drive.google.com/uc?id=1HnvjGJ4HmpS-0wndpH-Q8cKMwIwWkTUe'>

> **Retrieval in Action:**
>
> <img src='https://drive.google.com/uc?id=1ry2TWFsewwqYP3Lw9muuPmbyuQqXwnYV' width=800px>

> **Example workflow with embedding model:**
>
><br>
>
> <img src='https://drive.google.com/uc?id=1zTuMMX54L2HrnmCYktTxVfMVrkIz8w15' width=600px>

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M6_AST_01_RAG_with_OpenAI_LLMs" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")

    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://genai-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")

### Install Dependencies

In [ ]:
%%capture
!pip -q install openai
!pip -q install langchain-openai
!pip -q install langchain-core
!pip -q install langchain-community
!pip -q install sentence-transformers
!pip -q install langchain-huggingface
!pip -q install langchain-chroma
!pip -q install chromadb
!pip -q install pypdf

### Import Required Packages

In [ ]:
import os
import openai
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

#### **Provide your OpenAI API key**

In [ ]:
# Read OpenAI key from Colab Secrets

from google.colab import userdata

api_key = userdata.get('OPENAI_KEY')           # <-- change this as per your secret's name
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

### Load LLM

In [ ]:
# Load Model

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
# General query
response = llm.invoke("How to learn programming? give 5 points")
print(response.content)

### **Loading the documents**

[PDF Loader](https://python.langchain.com/docs/how_to/document_loader_pdf/)

In [ ]:
# UPLOAD the Docs first to this notebook, then run this cell

from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("/content/pca_d1.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),    # Loading duplicate documents on purpose
]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [ ]:
len(docs)        # 7 pages were there in total from above documents

In [ ]:
docs

In [ ]:
print(docs[0].page_content)

### **Splitting of document**

[Recursively split by character](https://python.langchain.com/docs/how_to/recursive_text_splitter/)

[Split by character](https://python.langchain.com/docs/how_to/character_text_splitter/)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)

print(len(splits))
print(len(splits[0].page_content) )
splits[0].page_content

In [ ]:
splits[0]

### **Embeddings**

Let's take our splits and embed them.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
embedding

### **Understanding similarity search with a toy example**

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like cats"
sentence3 = "the weather is ugly, too hot outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
len(embedding1), len(embedding2), len(embedding3)

In [ ]:
embedding1[:10]

In [ ]:
import numpy as np

def cosine_similarity(vector1, vector2):
    # Ensure that the vectors are numpy arrays
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)

    # Calculate the dot product of the vectors
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitude (norm) of the vectors
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)

    # Compute cosine similarity
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0  # Avoid division by zero
    return dot_product / (norm_vector1 * norm_vector2)


In [ ]:
cosine_similarity(embedding1, embedding2), cosine_similarity(embedding1, embedding3), cosine_similarity(embedding2, embedding3)

### **Vectorstores**

In [ ]:
from langchain_chroma import Chroma       # Light-weight and in memory

In [ ]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,                    # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory, # save the directory
)

In [ ]:
print(vectordb._collection.count()) # same as number of splits

### **Similarity Search in Vector store**

Algorithms for retrieving relevant chunks In Vector databases,

In vector databases, algorithms for retrieving relevant chunks to a query are often based on **similarity search techniques**, primarily using nearest neighbor search.

Here are some common approaches:

>**Approximate Nearest Neighbor (ANN) Search:** Vector databases frequently use ANN algorithms to improve efficiency when searching for vectors that
are close to the query vector.
>
>Popular **ANN** algorithms include:
>
>1. HNSW (Hierarchical Navigable Small World Graph): This is a graph-based approach that finds approximate nearest neighbors using a multi-
layered graph structure.
>
>2. Faiss: An open-source library developed by Facebook, which uses various algorithms for fast similarity search, such as Product Quantization and
Inverted File System (IVF).
>
>3. Annoy (Approximate Nearest Neighbors Oh Yeah): Developed by Spotify, it uses a forest of random projection trees for approximate nearest
neighbor search.


In [ ]:
question = "How does ensemble method works?"

In [ ]:
docs = vectordb.similarity_search(question, k=6)     # k --> No. of Document object to return

In [ ]:
print(len(docs))

for i in range(len(docs)):
    print(docs[i].page_content)
    print('='*140)

### **Edge cases where failure may happen**

1. Lack of Diversity : Semantic search fetches all similar documents, but does not enforce diversity.

    - Notice that we're getting duplicate chunks (because of the duplicate `ens_d2.pdf` in the index). `docs[0]` and `docs[1]` are indentical.

  **Addressing Diversity - MMR (Maximum Marginal Relevance)**

Maximum Marginal Relevance (MMR) is a method used to retrieve relevant items to a query while avoiding redundancy. It does this by ensuring a balance between relevancy and diversity in the items retrieved.

<img src='https://miro.medium.com/v2/resize:fit:828/format:webp/1*U-9mPt5tBfPBPrwC4_oD1w.png'>

In [ ]:
question = 'How ensemble method works?'
docs = vectordb.similarity_search(question, k=3)     # Without MMR

print(len(docs))

for i in range(len(docs)):
    print(docs[i].page_content)
    print('='*140)

**Example 1. Addressing Diversity - MMR-Maximum Marginal Relevance**

In [ ]:
docs_with_mmr = vectordb.max_marginal_relevance_search(question, k=3, fetch_k=6)   # With MMR

print(len(docs_with_mmr))

for i in range(len(docs_with_mmr)):
    print(docs_with_mmr[i].page_content)
    print('='*140)

2. Lack of specificity:  The question may be from a particular doc but answer may contain information from other doc.

  **Addressing Specificity: Working with metadata - Manually**

  **Working with metadata using self-query retriever - Automatically**

**Example 2. Addressing Specificity: Working with metadata - Manually**

In [ ]:
# Without metadata information
question = "What is variance?"

docs = vectordb.similarity_search(question, k=5)

for doc in docs:
    print(doc.metadata)    # metadata contains information about from which doc the answer has been fetched

We can filter the results based on metadata.

In [ ]:
# With metadata information
question = "what is the role of variance in pca?"
docs = vectordb.similarity_search(
    question,
    k=5,
    filter={"source":'/content/ens_d2.pdf'}     # manually passing metadata, using metadata filter.
)

for doc in docs:
    print(doc.metadata)

In [ ]:
# With metadata information + MMR

docs_with_mmr = vectordb.max_marginal_relevance_search(question,
                                                       k=2,
                                                       fetch_k=5,
                                                       filter={"source":'/content/ens_d2.pdf'}     # manually passing metadata, using metadata filter.
                                                       )

In [ ]:
for i in range(len(docs_with_mmr)):
    print(docs_with_mmr[i].page_content)
    print('='*140)

[**Addressing Specificity -Automatically: Working with metadata using self-query retriever**](https://python.langchain.com/docs/how_to/self_query/)

### **Additional tricks: Compression**

Another approach for improving the quality of retrieved docs is compression. Information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

[Contextual compression](https://python.langchain.com/docs/how_to/contextual_compression/) is meant to fix this.

## **Retrieval**

**[Vectorstore as a retriever](https://python.langchain.com/docs/how_to/vectorstore_retriever/)**

**Better Approach**

In [ ]:
# Without MMR
question = "What is principal component analysis?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(question)
docs

In [ ]:
# With MMR
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k":5})
docs = retriever.invoke(question)
docs

## **Augmentation**

In [ ]:
from langchain_core.prompts import PromptTemplate                                    # To format prompts
from langchain_core.output_parsers import StrOutputParser                            # to transform the output of an LLM into a more usable format
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough          # Required by LCEL (LangChain Expression Language)

In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

## **Creating final RAG Chain**

> <img src='https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F63f8a8482c9ec06a8d7d1041514f87c06dd108a9-3442x942.png&w=3840&q=75' width=1200px>

[[Image source](https://www.pinecone.io/learn/series/langchain/langchain-expression-language/)]

Above figure describes the LCEL flow using `RunnableParallel` and `RunnablePassthrough`.

A Runnable is a **unit of execution** in the LangChain framework. It represents a specific task or operation that can be performed.

Examples of Runnables include data transformations, computations, or any other operation that can be **expressed** in the LCEL(LangChain expression language).

[Runnable Lambdas](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) is a LangChain abstraction that allows us to turn Python functions into **pipe-compatible functions**, similar to the Runnable class.

[RunnablePassthrough](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) on its own allows you to pass inputs unchanged. This typically is **used in conjuction with [RunnableParallel](https://python.langchain.com/v0.1/docs/expression_language/interface/#parallelism)** to pass data through to a new key in the map.

The **RunnableParallel** object allows us to define multiple values and operations, and run them all in parallel.

The **RunnablePassthrough** object is used as a “passthrough” that takes any input to the current component ('retrieval' in above figure) and allows us to provide it in the component output via the “question” key or any other custom key.

In [ ]:
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 7, "fetch_k":15})
retriever

In [ ]:
retrieval = RunnableParallel(
    {
        "context": RunnablePassthrough(context= lambda x: x["question"] | retriever),
        "question": RunnablePassthrough()
        }
    )

In [ ]:
# RAG Chain

rag_chain = (retrieval                     # Retrieval
             | QA_PROMPT                   # Augmentation
             | llm                         # Generation
             | StrOutputParser()
             )

In [ ]:
response = rag_chain.invoke({"question": "What is PCA ?"})

response

In [ ]:
response = rag_chain.invoke({"question": "What is principal component analysis?"})

response

In [ ]:
response = rag_chain.invoke({"question": "How ensemble method works?"})

print(response)

In [ ]:
# For queries that is not in documents
response = rag_chain.invoke({"question": "Who is the CEO of OpenAI "})

print(response)

**To check what is being retreived from the retriever**

In [ ]:
chain_retriever = RunnablePassthrough() | retriever

In [ ]:
chain_retriever.invoke("What is principal component analysis?")

In [ ]:
chain_retriever.invoke("How ensemble method works?")

[**Details of Chroma through LangChain**](https://python.langchain.com/docs/integrations/vectorstores/chroma/)

## Reusing Vector DB

### **Download the vector DB**

In [ ]:
# Zip the entire folder
!zip -r /content/docs.zip /content/docs

In [ ]:
from google.colab import files
files.download("/content/docs.zip")

### **Upload the vector db from previous step and unzip**

In [ ]:
!unzip /content/docs.zip  -d /

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-small')

vectordb = Chroma(persist_directory = 'docs/chroma/',
                  embedding_function = embedding
                  )

### Please answer the questions below to complete the experiment:




In [ ]:
#@title Which of the following best describes the purpose of RAG? { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "To generate text without any external information", "To enhance text generation with relevant external data", "To solely retrieve information from databases", "To train models on unstructured data"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")